***

# 4. Introduction to temperature logs

This is a short introduction to a large topic. Given that geothermal seeks to mine heat energy, temperature logs are arguably the most important data we acquire. 

## 4.1 The temperature when?

When and under what well condition were the temperature logs acquired? 

## 4.2 Uses of temperature logs in geothermal

Feedzone identification (all raw well temperature logs)

Interpreting well/reservoir events, such as a change in the reservoir temperature with time such as heating after drilling or the initiation of a cool reservoir downflow (comparison of static temperature logs with time)

Constraining conceptual models and calibrating reservoir models (natural state temperature profiles)



***

# 5. Extract and evauluate temprature from completion test data

## 5.1 Use functions to import and mudge data

The flowrate and PTS data munging process in 1-overview.ipynb has been turned into specilised functions located in the functions.py file. We import and use that set of functions in the same way as any other module.

In [ ]:
import pandas as pd
from utilities import* # functions in the utilities.py file

Because the cell below includes all steps required to import and mudge the data (everything from notebook 1), it will take a little while to run. 

In [ ]:
flowrate = read_flowrate(r'Data-FlowRate.xlsx')
pts = read_pts(r'Data-PTS.xlsx')

In [ ]:
pts.info()

In [ ]:
flowrate.info()

In [ ]:
print(pts.shape)
pts.head()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(24,8),sharey=True)

spinner_scatter = ax1.scatter(pts.temp_degC, pts.depth_m, c = pts.timestamp, s = 5, linewidths = 0)
datetime_scatter = ax2.scatter(pts.datetime, pts.depth_m, c = pts.timestamp, s = 5, linewidths = 0)

ax3 = ax2.twinx()
ax3.plot(flowrate.datetime, flowrate.flow_tph, 
    c='k', linestyle = '-', linewidth = 3, alpha = 0.3, 
    label='Surface pump flowrate')

ax1.set_ylabel('Depth [m]')
ax1.set_xlabel('Temp [degC]')

ax2.set_xlabel('Time [hh:mm]')
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

ax3.set_ylabel('Flowrate [t/hr]')

ax1.set_ylim(1000,0)
ax1.set_xlim(0,200)

for ax in [ax1, ax2]:
    ax.grid()


In [ ]:
from ipywidgets import interactive, Layout, FloatSlider

min_timestamp = pts.timestamp.iloc[0]
max_timestamp = pts.timestamp.iloc[-1]

def subselect_plot(start_value, stop_value):
    f,ax = plt.subplots(1,1, figsize = (20,6))
    ax.plot(pts.timestamp, pts.depth_m, c = 'k')
    ymin = pts.depth_m.min()
    ymax = pts.depth_m.max() + 100
    ax.vlines(start_value, ymin, ymax, color='tab:green')
    ax.vlines(stop_value, ymin, ymax, color='tab:red')
    ax.set_ylim(pts.depth_m.max() + 100, 0)

result = interactive(subselect_plot,
         
         start_value = FloatSlider
         (
             value = (max_timestamp - min_timestamp)/3 + min_timestamp,
             description = 'start',
             min = min_timestamp, 
             max = max_timestamp, 
             step = 10, 
             continuous_update=True,
             layout = Layout(width='80%'),
             ),
          
          stop_value = FloatSlider
          (
             value = (max_timestamp - min_timestamp)/2 + min_timestamp, 
             description = 'stop',
             min = min_timestamp, 
             max = max_timestamp, 
             step = 10, 
             continuous_update=True,
             layout = Layout(width='80%')
             )
)

display(result);

# look into this for zoom https://towardsdatascience.com/how-to-produce-interactive-matplotlib-plots-in-jupyter-environment-1e4329d71651
# donest look to be working with ipywidgets

In [ ]:
print(
    'start =',result.children[0].value, ' which is', datetime.fromtimestamp(result.children[0].value), 
    '\n stop =', result.children[1].value, ' which is', datetime.fromtimestamp(result.children[1].value),
    )

### Some manual entry so the data selection is permanently recorded

As part of making this process repeatable and so that we can easily audit an existing analysis, the method at this point forces us to copy past the timestamps above to define the start and stop points for splicing the dataframe. This could be done just by calling the result.childern\[0\].value and result.childern\[1\].value objects, but these will change every time the sliders in the interactive plot change.

In [ ]:
# copy-paste the start and stop timestamps and 
# create an informative title for the data

start = 1607656164.448
stop = 1607656534.448
name = 'First heating run'

pts_first_heating_run = pts[
    (pts.datetime > datetime.fromtimestamp(start)) 
    & (pts.datetime < datetime.fromtimestamp(stop))
]

flowrate_first_heating_run = flowrate[
    (flowrate.datetime > datetime.fromtimestamp(start)) 
    & (flowrate.datetime < datetime.fromtimestamp(stop))
]

In [ ]:
pts_first_heating_run.shape

In [ ]:
overview_fig(pts_first_heating_run, flowrate_first_heating_run, title = name).show()

In [ ]:
pts.columns

## Import more temp data for comparison

In [ ]:
heating_37days = pd.read_csv('Data-heating37days.csv')

heating_37days['pressure_bara'] = heating_37days.pres_barg - 1

heating_37days.head(2)

In [ ]:
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates

fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(10,10),sharey=True)

ax1.scatter(pts.temp_degC, pts.depth_m, c = pts.timestamp, s = 5, linewidths = 0)
ax1.plot(pts_first_heating_run.temp_degC, pts_first_heating_run.depth_m, color = 'k', linestyle = ':')
ax1.set_title('Completion test temperature logs')

ax2.plot(pts_first_heating_run.temp_degC, pts_first_heating_run.depth_m, c='k', label = 'Heating 0 days - temperature')
ax2.plot(heating_37days.temp_degC, heating_37days.depth_m, c='r', label = 'Heating 37 days - temperature')

ax2.plot(pts_first_heating_run.pressure_bara, pts_first_heating_run.depth_m, ':', c='k', label = 'Heating 0 days - pressure')
ax2.plot(heating_37days.pressure_bara, heating_37days.depth_m, ':', c='r', label = 'Heating 37 days - pressure')

ax2.set_title('Heating temperature logs')
ax2.legend()
ax2.set_xlim(0,300)

ax1.set_ylim(1000,0)
ax1.set_ylabel('Depth [m]')
ax1.set_xlim(0,200)

for ax in [ax1,ax2]:
    ax.set_xlabel('Temp [degC]')
    ax.grid()


In [ ]:
# Add the visual representation of the caseing to these plots and the other by-depth plots
# add a fully heated T log (data is depth/temp/press, no time required) ... add pivot to the II introduction where we are talking about pressures ... say it is different in geothermal 
# add a BPD curve
# put pressure on a twinned x axis
# change pressure to a dashed line
# find a nicer colour for the stable temp run

PCS = 462.5 # mMD CHF, 13 3/8 production casing to surface
oh_liner = 10.75 # liner 
tol = 425 # mMD CHF


***

<p><center>© 2021 <a href="https://www.cubicearth.nz/">Irene Wallis</a> and <a href="https://www.linkedin.com/in/katie-mclean-25994315/">Katie McLean</a> <a href="https://creativecommons.org/licenses/by/4.0/"</a></center></p>

<p><center>Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at <a href="http://www.apache.org/licenses/LICENSE-2.0">http://www.apache.org/licenses/LICENSE-2.0</a></center></p>

<p><center>Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.</center></p>

***